In [1]:
import pandas as pd

In [27]:
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()


def query_db(query):
    """
    Execute a SQL query and return the results.
    """

    connection = psycopg2.connect(
        dbname='postgres',
        user=os.getenv('DB_ADMIN_NAME'),
        password=os.getenv('DB_ADMIN_PASSWORD'),
        host='clinical.cwpmisqy4ng3.us-east-1.rds.amazonaws.com',
        port=5432
    )
    
    with connection.cursor() as cursor:
        cursor.execute(query)
        results = cursor.fetchall()

    return results

In [16]:
# get all tables in the database
tables_query = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
tables = query_db(tables_query)
tables

[('follow_up',), ('clinical',), ('pathology_detail',), ('family_history',)]

In [62]:
columns_query = """
SELECT *
FROM information_schema.columns
WHERE table_name = 'clinical' AND column_name = 'treatments.number_of_cycles';
"""
columns = query_db(columns_query)
columns

[('postgres',
  'public',
  'clinical',
  'treatments.number_of_cycles',
  52,
  None,
  'YES',
  'double precision',
  None,
  None,
  53,
  2,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'postgres',
  'pg_catalog',
  'float8',
  None,
  None,
  None,
  None,
  '52',
  'NO',
  'NO',
  None,
  None,
  None,
  None,
  None,
  'NO',
  'NEVER',
  None,
  'YES')]

In [58]:
# select (demographic.ethnicity, demographic.gender) columns from clinical table
clinical_query = """
SELECT demographic.ethnicity
FROM clinical
WHERE demographic IS NOT NULL;
"""
clinical_data = query_db(clinical_query)
clinical_data

UndefinedTable: missing FROM-clause entry for table "demographic"
LINE 2: SELECT demographic.ethnicity
               ^


In [44]:
query_db("""
            SELECT column_name
            FROM information_schema.columns
            WHERE table_name = 'clinical'
        """
        )

[('project.project_id',),
 ('cases.case_id',),
 ('cases.submitter_id',),
 ('family_histories.relationship_primary_diagnosis',),
 ('family_histories.relative_with_cancer_history',)]

In [19]:
import boto3

dynamodb = boto3.resource('dynamodb', 
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_REGION')
    )

table = dynamodb.Table('biospecimen')

response = table.query(
    KeyConditionExpression=boto3.dynamodb.conditions.Key('case_id').eq('001ad307-4ad3-4f1d-b2fc-efc032871c7e')
)

# Process Results
for item in response['Items']:
    print(item)

{'samples': [{'sample_type_id': '01', 'tumor_descriptor': 'Primary', 'sample_id': '6bddf3fb-b808-46ff-8207-ce510315448d', 'submitter_id': 'TCGA-HT-A614-01Z', 'sample_type': 'Primary Tumor', 'created_datetime': '2018-05-17T12:24:55.491180-05:00', 'oct_embedded': 'No', 'specimen_type': 'Solid Tissue', 'days_to_sample_procurement': '0', 'updated_datetime': '2023-11-08T11:18:34.935258-06:00', 'state': 'released', 'is_ffpe': 'true', 'preservation_method': 'FFPE', 'tissue_type': 'Tumor', 'portions': [{'analytes': [], 'portion_id': '09243ea2-060a-5517-b120-af16d16dd4de', 'slides': [{'submitter_id': 'TCGA-HT-A614-01Z-00-DX1', 'section_location': 'Not Reported', 'state': 'released', 'slide_id': 'e91182f1-3962-47e9-81bf-a7b3f6e45583', 'updated_datetime': '2018-08-23T19:39:28.814253-05:00', 'created_datetime': '2018-05-17T13:50:09.687385-05:00'}]}]}, {'sample_type_id': '10', 'tumor_descriptor': 'Not Applicable', 'sample_id': '70dba908-6b41-4081-b0bf-4b46a922ae42', 'submitter_id': 'TCGA-HT-A614-10

In [ ]:
# Visualize the types of biospecimens (e.g., Tumor, Blood Derived Normal) using a pie or donut chart. Include insights on preservation methods (FFPE vs. fresh) and tissue types.

response = tab